In [1]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from qiskit.circuit.library import PauliFeatureMap, ZFeatureMap, ZZFeatureMap
from qiskit_machine_learning.kernels import QuantumKernel
import qiskit
from cuquantum import *
import nvtx

In [2]:
traindata = pd.read_csv('../data/fashion-mnist-data/fashion-mnist_train.csv')
filt = (traindata['label'] < 2)
two_class = traindata.loc[filt]

In [3]:
def fashion_train_data(n_dim = 2,datasize = 2):
    data_train  = two_class.iloc[:,1:785] / 255.0
    label_train = pd.DataFrame([two_class.iloc[:,0]]).T
    l_train=pd.DataFrame([two_class.iloc[:,0]]).T
    X_train, X_val, Y_train, Y_val = train_test_split(data_train, l_train, test_size = 0.25, random_state=255)
    X_train = StandardScaler().fit_transform(X_train)
    X_val   = StandardScaler().fit_transform(X_val)
    pca = PCA(n_components=n_dim).fit(X_train)
    X_train = pca.transform(X_train)
    X_train = X_train[:datasize]
    Y_train = np.array(Y_train).ravel()
    Y_train = Y_train[:datasize]  
    return X_train, Y_train

In [4]:
def qc_qiskit2cuquantum(x_t1, x_t2, n_dim):
    zz_map = ZZFeatureMap(feature_dimension=n_dim, reps=1, entanglement="linear", insert_barriers=True)
    zz_kernel = QuantumKernel(feature_map=zz_map)  
    zz_circuit = zz_kernel.construct_circuit(x_t1,x_t2,measurement=False,is_statevector_sim=False)
    converter = CircuitToEinsum(zz_circuit, backend='cupy')
    a = str(0).zfill(n_dim)
    exp, oper = converter.amplitude(a)  
    return zz_circuit, exp, oper

In [5]:
def get_path(exp, oper):
    qsvm_path, info = contract_path(exp,*oper)
    return qsvm_path

In [6]:
def get_kernel_matrix(x_t,datasize,n_dim,path):
    kernel_matrix = np.zeros((datasize,datasize))
    for i1 in range(2,datasize+1):
        for i2 in range(1,i1):
            qc_tmp, exp_tmp, oper_tmp = qc_qiskit2cuquantum(x_t[i1-1],x_t[i2-1],n_dim)
            amp = contract(exp_tmp,*oper_tmp, optimize = {'path' : path})
            amp_tmp = np.round(np.sqrt(np.square(amp.real)+np.square(amp.imag)),5)
            kernel_matrix[i1-1][i2-1] = amp_tmp
    kernel_matrix = kernel_matrix+kernel_matrix.T+np.diag(np.ones((datasize)))
    return kernel_matrix  

In [7]:
n_dim, datasize  = 2, 2
x_t, y_t = fashion_train_data(n_dim, datasize)
qc, exp, oper = qc_qiskit2cuquantum(x_t[0], x_t[0], n_dim)
path = get_path(exp, oper)
get_kernel_matrix(x_t,datasize,n_dim,path)

array([[1.     , 0.55358],
       [0.55358, 1.     ]])